In [1]:
import tensorflow as tensorflow
import pandas as pd 
import h5py
import os

# Preparing data

In [2]:
TRAIN_PATH = 'D:/Projects/HealthStarRating/Food-data/Images/Train'
VALIDATE_PATH = 'D:/Projects/HealthStarRating/Food-data/Images/Validate'
TEST_PATH = 'D:/Projects/HealthStarRating/Food-data/Images/Test'

MODELS_PATH = 'D:/Projects/HealthStarRating/Models'
BEST_MODEL = 'fine_tune_model_best.hdf5'

IMAGE_SIZE = (300, 300)
BATCH_SIZE = 128

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
validate_generator = ImageDataGenerator(rescale=1./255)
test_generator = ImageDataGenerator(rescale=1./255)

In [4]:
generated_train_data = train_generator.flow_from_directory(TRAIN_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
generated_validate_data = validate_generator.flow_from_directory(VALIDATE_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
generated_test_data = test_generator.flow_from_directory(TEST_PATH, target_size=IMAGE_SIZE)

Found 36254 images belonging to 40 classes.
Found 4521 images belonging to 40 classes.
Found 4560 images belonging to 40 classes.


# EVALUATION

In [5]:
from keras.models import load_model
from keras.optimizers import SGD
from keras.metrics import TopKCategoricalAccuracy
from tqdm.notebook import tqdm
validate_results = {}
test_results = {}

In [6]:
for folder in tqdm(os.listdir(MODELS_PATH)):
    model_folder = os.path.join(MODELS_PATH, folder)
    if BEST_MODEL in os.listdir(model_folder):
        print('\n========== Evaluate', folder, 'Model ==========')
        model = load_model(os.path.join(model_folder, BEST_MODEL))
        model.compile(
            optimizer = SGD(learning_rate=1e-4, momentum=0.9),
            loss = 'categorical_crossentropy', 
            metrics = [
                'accuracy', 
                TopKCategoricalAccuracy(k=3, name='top_3_accuracy'), 
                TopKCategoricalAccuracy(k=5, name='top_5_accuracy')
            ]
        )
        print('Validate dataset:')
        validate_results[folder] = model.evaluate(generated_validate_data)
        print('Test dataset:', )
        test_results[folder] = model.evaluate(generated_test_data)

  0%|          | 0/7 [00:00<?, ?it/s]


========== Evaluate InceptionResNetV2 Model ==========


Validate dataset:


36/36 [==============================] - 328s 9s/step - loss: 0.5600 - accuracy: 0.8326 - top_3_accuracy: 0.9509 - top_5_accuracy: 0.9790
Test dataset:
143/143 [==============================] - 332s 2s/step - loss: 0.5962 - accuracy: 0.8213 - top_3_accuracy: 0.9436 - top_5_accuracy: 0.9717

========== Evaluate InceptionV3 Model ==========
Validate dataset:
36/36 [==============================] - 121s 3s/step - loss: 0.6209 - accuracy: 0.8160 - top_3_accuracy: 0.9427 - top_5_accuracy: 0.9730
Test dataset:
143/143 [==============================] - 122s 849ms/step - loss: 0.6377 - accuracy: 0.8099 - top_3_accuracy: 0.9399 - top_5_accuracy: 0.9673

========== Evaluate ResNet101V2 Model ==========
Validate dataset:
36/36 [==============================] - 332s 9s/step - loss: 0.5699 - accuracy: 0.8487 - top_3_accuracy: 0.9562 - top_5_accuracy: 0.9783
Test dataset:
143/143 [==============================] - 345s

In [7]:
validate_report = pd.DataFrame.from_dict(validate_results, orient='index').iloc[:, 1:]
validate_report.columns = ['Accuracy', 'Top 3 Accuracy', 'Top 5 Accuracy']
validate_report.sort_values(by=['Accuracy'], ascending=False)

,Accuracy,Top 3 Accuracy,Top 5 Accuracy
ResNet152V2,0.858881,0.958637,0.983411
ResNet101V2,0.848706,0.956204,0.978323
Xception,0.844282,0.957089,0.976775
ResNet50V2,0.841407,0.951781,0.977217
InceptionResNetV2,0.832559,0.950896,0.978987
InceptionV3,0.815970,0.942712,0.973015


In [8]:
test_report = pd.DataFrame.from_dict(test_results, orient='index').iloc[:, 1:]
test_report.columns = ['Accuracy', 'Top 3 Accuracy', 'Top 5 Accuracy']
test_report.sort_values(by=['Accuracy'], ascending=False)

,Accuracy,Top 3 Accuracy,Top 5 Accuracy
ResNet152V2,0.859211,0.959430,0.978728
ResNet50V2,0.846491,0.955482,0.978289
Xception,0.840570,0.949781,0.976316
ResNet101V2,0.837719,0.953289,0.978070
InceptionResNetV2,0.821272,0.943640,0.971711
InceptionV3,0.809868,0.939912,0.967325
